# Baseline Model

This notebook will contain the baseline MVP model as well as continued data exploration. Though this notebook is for developing the first model, there will be multiple model iterations and experimentation with various features and methods for dealing with missing values in weather data.

### Plan
- Import libraries/energy and weather dataframes
- Data Exploration
- Initial weather data aggregation and imputing
- First Simple Model

## NOTE
Based on research into the current model's being used to accomplish this project's task (found in references/ directory) current weather observations can be used to accurately forecast only a few hours in the future. Extended forecasts (three hours +) require weather forecasts to be more accurate. For that reason, this MVP model, which only uses current weather observations, will forecast solar energy production three hours in advance. In later iterations of the model I will include weather forecasts and aim to predict solar energy output twenty-four hours in advance.

## Import libraries and data

In [1]:
# Import standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import Machine Learning Models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

# Import Data Preprocessing
from sklearn.preprocessing import StandardScaler

# Import ML Metrics
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score

# Cross Validation
from sklearn.model_selection import train_test_split, cross_val_score, KFold

# Use functions from .py file
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import src.data_gathering as dg


I want to have a function that calculates the adjusted R squared score. Since I will be experimenting with number of features, I want my metrics to reflect the quality of the model and not just the number of features being used to train it.

In [2]:
def adjusted_r_squared(r, n, p):
    adjusted_score = (1 - (1-r)) * ((n-1)/(n-p-1))
    return adjusted_score

Next I will get the base hourly energy and weather dataframes using the functions from the data_gathering.py file in the src/ directory.

In [3]:
energy = dg.energy_data()
weather = dg.weather_data()

In [4]:
energy.head()

,nexus_meter,inverter
time,,
2018-01-30 00:00:00,0.0,0.0
2018-01-30 01:00:00,0.0,0.0
2018-01-30 02:00:00,0.0,0.0
2018-01-30 03:00:00,0.0,0.0
2018-01-30 04:00:00,0.0,0.0


In [5]:
weather.head()

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,cloud_coverage
DATE,,,,,,,,,
2018-01-29 00:23:00,30.34,28,30,92,NaN,29.51,5.00,14,80.0
2018-01-29 00:42:00,30.35,28,30,92,NaN,29.52,6.00,13,80.0
2018-01-29 00:51:00,30.34,28,30,93,NaN,29.51,6.00,15,80.0
2018-01-29 00:53:00,30.34,28,30,92,30.36,29.51,6.00,15,80.0
2018-01-29 01:53:00,30.34,28,30,92,30.35,29.51,3.00,10,80.0


Both dataframes were imported correctly. I would like to investigate the weather data more. In particular, is there any pattern for missing data (occuring at certain times or in relation to other features) and also the occurence of the '*' symbol in a few of the columns.

## Data Exploration

First I would like to explore the missing values in the weather dataframe and see if there is any recurring pattern for missing data. This will hopefully provide some insight into the best way to impute these missing values.

In [6]:
weather.shape

(30124, 9)

In [7]:
weather.isna().sum()

HourlyAltimeterSetting        939
HourlyDewPointTemperature    1015
HourlyDryBulbTemperature      934
HourlyRelativeHumidity       1015
HourlySeaLevelPressure       8449
HourlyStationPressure        1026
HourlyVisibility              937
HourlyWindSpeed              1019
cloud_coverage               1018
dtype: int64

The majority of the columns are only missing about 1,000 observations. 'HourlySeaLevelPressure' is the exception and is missing nearly a third of it's observations. One thing to note is that this dataframe has not been aggregated hourly yet. I know from the notebook 02_explore that when I perform hourly aggregation, there are only a hundred or so missing values in each column. This means that for the most part, at least one observation within each our contains data. This is good because it is unlikely weather conditions will change dramatically within a single hour, so even one observation in a given hour still provides some level of accuracy of what the weather was like during that entire hour.

#### It is likely I will impute the missing data AFTER aggregating, but I want to explore the missing values at a more granular level before doing this.

The first thing I want to do is see if the missing data is scattered throughout the dataframe, or if for the most part columns are missing data at the same index. To do this I am going to create a dictionary with the key as the column names and the values as a list of the indices with missing values for it's respectice column.

In [8]:
cols = weather.columns
missing = {}

In [9]:
for col in cols:
    missing[col] = weather[weather[col].isna() == True].index

In [10]:
missing

{'HourlyAltimeterSetting': DatetimeIndex(['2018-01-29 23:59:00', '2018-01-30 23:59:00',
                '2018-01-31 23:59:00', '2018-01-31 23:59:00',
                '2018-02-01 23:59:00', '2018-02-02 23:59:00',
                '2018-02-03 23:59:00', '2018-02-04 23:59:00',
                '2018-02-05 23:59:00', '2018-02-06 23:59:00',
                ...
                '2020-07-11 23:59:00', '2020-07-12 23:59:00',
                '2020-07-13 23:59:00', '2020-07-14 23:59:00',
                '2020-07-15 23:59:00', '2020-07-16 23:59:00',
                '2020-07-17 23:59:00', '2020-07-18 23:59:00',
                '2020-07-19 23:59:00', '2020-07-20 23:59:00'],
               dtype='datetime64[ns]', name='DATE', length=939, freq=None),
 'HourlyDewPointTemperature': DatetimeIndex(['2018-01-29 23:59:00', '2018-01-30 23:59:00',
                '2018-01-31 23:59:00', '2018-01-31 23:59:00',
                '2018-02-01 23:59:00', '2018-02-02 23:59:00',
                '2018-02-03 23:59:00', '20

Just looking at the output above, it seems like practically all of the missing values are at the same time (23:59:00). Excpet for 'HourlySeaLevelPressure' which has many more missing values. This means the majority of missing values for each column are at the same time. Also, the missing values are at 12am, which is the middle of the night. There is no energy production at this time so i may be able to drop these rows altogether. I want to look at these indices in the weather dataframe

In [11]:
weather[weather['cloud_coverage'].isna() == True]

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,cloud_coverage
DATE,,,,,,,,,
2018-01-29 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-30 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-31 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-31 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-01 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2020-07-16 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-17 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-18 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The missing valuesa are at the same time every day. This data is not very useful for me since there is no energy production at this time of the night anyways. I do want to double check this before dropping these rows.

In [12]:
energy[energy.index.hour == 23]

,nexus_meter,inverter
time,,
2018-01-30 23:00:00,-52.50,0.0
2018-01-31 23:00:00,-42.69,0.0
2018-02-01 23:00:00,-49.70,0.0
2018-02-02 23:00:00,-46.00,0.0
2018-02-03 23:00:00,-42.10,0.0
...,...,...
2020-07-16 23:00:00,-55.30,0.0
2020-07-17 23:00:00,-54.40,0.0
2020-07-18 23:00:00,-54.30,0.0


These time indices do not produce any solar energy output, so they will be excluded from the model anyways. I am going to drop the rows from the weather dataframe at this time since they are all missing data. I need to be sure I don't drop any rows that may be missing data at other times though, so i need to pass in one more filtering condition and make sure the time index is correct

In [13]:
weather[(weather['cloud_coverage'].isna() == True) & (weather.index.hour == 23)]

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,cloud_coverage
DATE,,,,,,,,,
2018-01-29 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-30 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-31 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-31 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-01 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2020-07-16 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-17 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-18 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are a few other rows that are missing 'cloud_coverage' values. I want to check 'HourlyAltimeterSetting', which has the same number of missing values as this filtered dataframe above.

In [14]:
weather[(weather['HourlyAltimeterSetting'].isna() == True) & (weather.index.hour == 23)]

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,cloud_coverage
DATE,,,,,,,,,
2018-01-29 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-30 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-31 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-31 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-01 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2020-07-16 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-17 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-18 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
weather[(weather.index.hour == 23) & (weather.index.minute == 59)].isna().sum()

HourlyAltimeterSetting       932
HourlyDewPointTemperature    933
HourlyDryBulbTemperature     932
HourlyRelativeHumidity       933
HourlySeaLevelPressure       935
HourlyStationPressure        932
HourlyVisibility             932
HourlyWindSpeed              932
cloud_coverage               932
dtype: int64

All of the columns are missing the vast majority of values at this time index. Since this time is not useful for my analysis, I am going to drop this time index.

In [16]:
to_drop = weather[(weather.index.hour == 23) & (weather.index.minute == 59)].index

In [17]:
weather.drop(to_drop, axis=0, inplace=True)

In [18]:
weather.isna().sum()

HourlyAltimeterSetting          7
HourlyDewPointTemperature      82
HourlyDryBulbTemperature        2
HourlyRelativeHumidity         82
HourlySeaLevelPressure       7514
HourlyStationPressure          94
HourlyVisibility                5
HourlyWindSpeed                87
cloud_coverage                 86
dtype: int64

That took care of pretty much all of the missing data besides 'HourlySeaLevelPressure'. I want to look closer at that column now.

In [19]:
missing_hours = weather[weather['HourlySeaLevelPressure'].isna() == True].index.hour

In [20]:
np.unique(missing_hours)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

This column is missing data at all time indices. There is a chance I may not use this column (if it is not a good predictor of solar energy production), so i am going to leave it for now and figure out how to deal with these missing values later on. The last thing I want to do before I begin modelling is explore the occurence of '*' throughout the dataframe. Again, according to the documentation (found in references/ directory), this value means that the value is included in the following measurement because time distribution is unknown. So I may be able to drop these rows. A problem occurs however, if this value appears in subsequent rows, because the data that is supposed to be included in the following measurement is passed along without ever actually appearing.

In [21]:
star_present = {}
for col in cols:
    star_present[col] = weather[weather[col] == '*'].index

In [22]:
star_present

{'HourlyAltimeterSetting': DatetimeIndex([], dtype='datetime64[ns]', name='DATE', freq=None),
 'HourlyDewPointTemperature': DatetimeIndex(['2018-03-28 10:53:00', '2020-06-23 11:53:00',
                '2020-06-24 11:58:00'],
               dtype='datetime64[ns]', name='DATE', freq=None),
 'HourlyDryBulbTemperature': DatetimeIndex(['2018-03-28 10:53:00', '2020-06-23 11:53:00',
                '2020-06-24 11:58:00'],
               dtype='datetime64[ns]', name='DATE', freq=None),
 'HourlyRelativeHumidity': DatetimeIndex(['2018-03-28 10:53:00', '2020-06-23 11:53:00',
                '2020-06-24 11:58:00'],
               dtype='datetime64[ns]', name='DATE', freq=None),
 'HourlySeaLevelPressure': DatetimeIndex([], dtype='datetime64[ns]', name='DATE', freq=None),
 'HourlyStationPressure': DatetimeIndex([], dtype='datetime64[ns]', name='DATE', freq=None),
 'HourlyVisibility': DatetimeIndex(['2018-03-27 11:53:00', '2019-03-15 08:01:00',
                '2019-03-15 08:10:00', '2019-03-15 08:38

The columns 'HourlyDewPointTemperature', 'HourlyDryBulbTemperature', and 'HourlyRelativeHumidity' all have three occurences of the '*' value and all at the same index. 'HourlyVisibility' has 8 occurences and at different time indices than the other three columns. The three occurences all occur at different hours of different days, so i can drop those rows once I make sure there is another observaton at that hour to represent the weather conditions at that time period. If there is not, I will convert '*' value to null and impute it with other missing values.

In [23]:
weather[weather.index.hour == 10]['2018-03-28']

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,cloud_coverage
DATE,,,,,,,,,
2018-03-28 10:53:00,29.96,*,*,*,NaN,29.14,5.00,5,80.0


In [24]:
weather[weather.index.hour == 11]['2020-06-23']

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,cloud_coverage
DATE,,,,,,,,,
2020-06-23 11:53:00,29.85,*,*,*,NaN,29.03,10.00,17,20.0


In [25]:
weather[weather.index.hour == 11]['2020-06-24']

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,cloud_coverage
DATE,,,,,,,,,
2020-06-24 11:53:00,29.93,56,75,52,29.92,NaN,10.00,7,NaN
2020-06-24 11:58:00,29.93,*,*,*,NaN,29.11,10.00,0,40.0


The occurence on '2020-06-24' has another value that can represent the values at that time period, the other two do not. So I will drop the last one and convert the other to nan.

In [26]:
weather.drop(weather[(weather.index.hour == 11) & (weather.index.minute == 58)]['2020-06-24'].index, axis=0, inplace=True)

In [27]:
weather['HourlyDewPointTemperature'].loc[weather['HourlyDewPointTemperature'] == '*'] = np.nan
weather['HourlyDryBulbTemperature'].loc[weather['HourlyDryBulbTemperature'] == '*'] = np.nan
weather['HourlyRelativeHumidity'].loc[weather['HourlyRelativeHumidity'] == '*'] = np.nan

In [28]:
weather[weather.index.hour == 10]['2018-03-28']

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,cloud_coverage
DATE,,,,,,,,,
2018-03-28 10:53:00,29.96,NaN,NaN,NaN,NaN,29.14,5.00,5,80.0


In [29]:
weather[weather.index.hour == 11]['2020-06-23']

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,cloud_coverage
DATE,,,,,,,,,
2020-06-23 11:53:00,29.85,NaN,NaN,NaN,NaN,29.03,10.00,17,20.0


Now that those values are taken care of, I can look closer at the star values from the 'HourlyVisibility' column

In [30]:
weather[weather['HourlyVisibility'] == '*']

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,cloud_coverage
DATE,,,,,,,,,
2018-03-27 11:53:00,29.99,56,56,100,30,29.17,*,14,80.0
2019-03-15 08:01:00,29.96,33,34,97,NaN,29.14,*,21,80.0
2019-03-15 08:10:00,29.96,33,34,97,NaN,29.14,*,22,80.0
2019-03-15 08:38:00,29.97,33,34,97,NaN,29.15,*,20,80.0
2019-03-15 08:46:00,29.97,33,35,93,NaN,29.15,*,23,80.0
2019-03-15 08:51:00,29.98,34,36,93,NaN,29.16,*,25,80.0
2019-03-15 08:53:00,29.98,34,35,96,29.99,29.16,*,22,80.0
2019-12-01 21:28:00,29.72,34,35,96,NaN,28.91,*,15,80.0


I am going to convert these to nan as well and impute them with the other missing values. The problem is that all 'HourlyVisibility' values on '2019-03-15' at hour 8 are unknown. When i aggregate this dataframe though, I will be able to impute these values with values before it, so it shouldn't be a problem.

In [31]:
weather['HourlyVisibility'].loc[weather['HourlyVisibility'] == '*'] = np.nan

In [32]:
weather[weather['HourlyVisibility'] == '*']

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,cloud_coverage
DATE,,,,,,,,,


Great now those values are taken of, I can now aggregate the dataframe and begin exploring the relationship between different features and the target value and then begin modeling.

In [33]:
weather = weather.astype(float)

ValueError: could not convert string to float: '0.50s'

For some reason the weather_data function missed the 's' present in 'HourlyVisibility'. I will fix that function later, for now I will remove it manually.

In [34]:
weather['HourlyVisibility'] = dg.remove_letter_from_column(weather, 'HourlyVisibility', 's')

Now that the only thing I have to worry about is missing values. I would like to impute the missing values before I aggregate because I think the aggregation would be more accurate if the values were already imputed. Now I just need to determine the best way to fill the missing values. There are a few approaches I could take, none of which are perfect, but I can experiment with different imputation strategies during model iterations to see which works best. Two ideas come to mind: 

- Using the past values of a given column to predict the missing values. For example if 'HourlyVisibility' was missing a value at 3 pm, I could take the average of the values before it and use that to fill the missing value. The problem with this is that there are multiple occurences of successive missing values, so i would be using imputed values to fill subsequent missing values. Since the weather conditions don't change too drastically over a short time period, I don't think this would be too big a problem.

- A second method would be using K Nearest Neighbors to impute the missing values. I could use the other weather conditions at a given time to impute the missing value of a certain feature. The problem with this strategy occurs when there are missing values for multiple features in a given row, which happens a few times.

I think for now the best way to go about it would be combining the two. 'HourlySeaLevelPressure' has too many missing values for me to be comfortable using the first method. So I will use the first method for all the other columns, and use KNN to impute the missing values for 'HourlySeaLevelPressure'

So I want to create two imputer objects, one for each method above.

In [48]:
from sklearn.base import BaseEstimator

In [47]:
# Imputer object 1
class PastAverageImputer(BaseEstimator):

    def __init__(self, target, time_frame=3):
        
        self.target = target
        self.time_frame = time_frame

    def fit(self, X, y=None):
        

After a little research, I found an sklearn imputer that I believe would work well for the circumstances of the current dataframe. IterativeImputer is an experimental imputation strategy, so I need to enable it, but it's strategy is just what I need. What it does is perform a linear regression on every missing value, using the other columns as features, in a round robin fashion. The imputation_order parameter let's you decide which column's missing value get's filled first. By default it goes in ascending order, so the column with the least amount of missing values is filled first, then the second, and so on. I left this parameter to be default.

In [50]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

I want to create a copy of my weather dataframe in case the imputation alters it in any way.

In [51]:
weather_copy = weather.copy()

In [52]:
imp = IterativeImputer(random_state=42)

In [53]:
weather_imputed = imp.fit_transform(weather_copy)

In [55]:
weather_imputed

array([[30.34, 28.  , 30.  , ...,  5.  , 14.  , 80.  ],
       [30.35, 28.  , 30.  , ...,  6.  , 13.  , 80.  ],
       [30.34, 28.  , 30.  , ...,  6.  , 15.  , 80.  ],
       ...,
       [30.04, 69.  , 72.  , ..., 10.  ,  0.  ,  0.  ],
       [30.04, 68.  , 71.  , ..., 10.  ,  0.  ,  0.  ],
       [30.02, 68.  , 71.  , ..., 10.  ,  0.  ,  0.  ]])

The imputed data is return as a numpy array, so I need to put it back in a dataframe. I can take the index and columns from the weather dataframe.

In [56]:
weather_imputed_df = pd.DataFrame(index=weather_copy.index, columns=weather_copy.columns, data=weather_imputed)

In [58]:
weather_imputed_df.isna().sum()

HourlyAltimeterSetting       0
HourlyDewPointTemperature    0
HourlyDryBulbTemperature     0
HourlyRelativeHumidity       0
HourlySeaLevelPressure       0
HourlyStationPressure        0
HourlyVisibility             0
HourlyWindSpeed              0
cloud_coverage               0
dtype: int64

No more missing values! I may still experiment with different imputation methods in further model iterations, but for now I think this works great. Now I can begin creating my first basic model.

## First Simple Model.

To begin, I can aggregate the weather data into an hourly frequency.

In [62]:
weather_hourly = weather_imputed_df.resample('H').mean()

In [63]:
weather_hourly.head()

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,cloud_coverage
DATE,,,,,,,,,
2018-01-29 00:00:00,30.3425,28.0,30.0,92.25,30.364994,29.5125,5.75,14.25,80.0
2018-01-29 01:00:00,30.3400,28.0,30.0,92.00,30.350000,29.5100,3.00,10.00,80.0
2018-01-29 02:00:00,30.3300,28.0,30.0,92.00,30.340000,29.5000,5.00,13.00,80.0
2018-01-29 03:00:00,30.3300,28.0,30.0,92.00,30.340000,29.5000,6.00,13.00,80.0
2018-01-29 04:00:00,30.3300,28.0,30.0,92.50,30.351525,29.5000,6.00,10.50,80.0


Next, I combine the weather and energy data into a single dataframe, incorporating a three hour lag between weather data and energy data. I can do this by using .shift() on the weather data when concatenating.

In [64]:
model_df = pd.concat([energy, weather_hourly.shift(3)], axis=1)

In [65]:
model_df.head()

,nexus_meter,inverter,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,cloud_coverage
DATE,,,,,,,,,,,
2018-01-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-29 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-29 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-29 03:00:00,NaN,NaN,30.3425,28.0,30.0,92.25,30.364994,29.5125,5.75,14.25,80.0
2018-01-29 04:00:00,NaN,NaN,30.3400,28.0,30.0,92.00,30.350000,29.5100,3.00,10.00,80.0


the inverter column from the energy data provides no valuable information so I am going to drop it. I am also going to drop the first 24 rows of the dataframe as they have no energy data.

In [69]:
model_df.drop('inverter', axis=1, inplace=True)

In [71]:
model_df = model_df['2018-01-30 1:00:00':]

In [72]:
model_df.head(10)

,nexus_meter,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,cloud_coverage
DATE,,,,,,,,,,
2018-01-30 01:00:00,0.0,30.48,13.0,17.0,84.0,30.52,29.65,10.0,6.0,0.0
2018-01-30 02:00:00,0.0,30.48,13.0,17.0,84.0,30.51,29.65,10.0,5.0,0.0
2018-01-30 03:00:00,0.0,30.46,14.0,17.0,88.0,30.50,29.63,10.0,6.0,0.0
2018-01-30 04:00:00,0.0,30.47,11.0,15.0,84.0,30.50,29.64,10.0,6.0,0.0
2018-01-30 05:00:00,0.0,30.47,11.0,14.0,88.0,30.50,29.64,10.0,5.0,0.0
2018-01-30 06:00:00,0.0,30.46,11.0,14.0,88.0,30.49,29.63,10.0,5.0,0.0
2018-01-30 07:00:00,0.0,30.45,10.0,13.0,88.0,30.49,29.62,10.0,0.0,0.0
2018-01-30 08:00:00,0.0,30.47,11.0,14.0,88.0,30.51,29.64,10.0,6.0,0.0
2018-01-30 09:00:00,0.0,30.50,11.0,13.0,92.0,30.54,29.67,10.0,6.0,0.0


I have my dataframe formatted properly, now I can perform a train test split and run it through a dummy_regressor from sklearn to obtain a few baseline metrics that I compare future model iterations against.

In [73]:
model_df.shape

(21671, 10)

I want to perform a 80/20 train test split. That will give me about 17,000 rows for training and the remainder for testing.

In [75]:
# Target
y = model_df['nexus_meter']
# Features
X = model_df.drop('nexus_meter', axis=1)

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.2)

For this basline model, I will use sklearn's dummy_regressor.

In [78]:
from sklearn.dummy import DummyRegressor

In [104]:
dr = DummyRegressor(strategy='mean')

In [105]:
scaler = StandardScaler()

In [106]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [107]:
dr.fit(X_train_scaled, y_train)

DummyRegressor()

In [108]:
train_preds = dr.predict(X_train_scaled)

In [109]:
np.sqrt(mse(y_train, train_preds))

5170.219268550485

In [110]:
test_preds = dr.predict(X_test_scaled)

In [111]:
np.sqrt(mse(y_test, test_preds))

5206.865865892132

So the baseline model produced an RMSE of about 5,200 for both the training and test set. That is pretty bad and should only go up from there. I'm curious how much better it would perform when looking at only the hour's that generate energy (for now we will consider this 5am to 9pm)

In [116]:
y2 = model_df['nexus_meter']
X2 = model_df.drop('nexus_meter', axis=1)

In [117]:
y2 = y2[(y2.index.hour >= 5) & (y2.index.hour <= 21)]
X2 = X2[(X2.index.hour >= 5) & (X2.index.hour <= 21)]

In [118]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state=42, test_size=.2)

In [119]:
X_train2_scaled = imp.fit_transform(X_train2)

In [120]:
dr2 = DummyRegressor()

In [122]:
dr2.fit(X_train2_scaled, y_train2)

DummyRegressor()

In [123]:
train_preds2 = dr2.predict(X_train2_scaled)

In [124]:
np.sqrt(mse(y_train2, train_preds2))

5642.56604005931

Unexpectedly, the subset model actually performed worse than the entire dataset. I am thinking this is because of how the dummy regressor makes it's predictions (mean of target), and when I run a legitamite model on it in the next notebook, I still think the subset will perform better.